In [ ]:
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
from user_agent import generate_user_agent
import time
import datetime

# Kilistázom az oldalakról az albik linkjeit

In [ ]:
url1="https://www.alberlet.hu/kiado_alberlet/page:"
url2="/ingatlan-tipus:lakas/kerulet:i+ii+iii+iv+v+vi+vii+viii+ix+x+xi+xii+xiii+xiv+xv+xvi+xvii+xviii+xix+xx+xxi+xxii+xxiii/megye:budapest/keres:normal/limit:24"
links=[]
start_time=time.time()
for page in range(1, 120):
    try:       
        r=requests.get(url1+str(page)+url2, timeout=5)
        if r.status_code == 200:
            print('Sikeres kapcsolat!')
            main_soup=BeautifulSoup(r.content, 'html.parser')

            #az oldal tárolja str ként az oldal számot
            #ha a linkbe nagyobb számot irok be akkor az oldalon tárolt szám nem változik
            #igy meg tudom mondani, hogy az utolsó oldalon vagyok-é
            page_nr=main_soup.find('li', {"class", "current"}).text    

            if page_nr == str(page):

                list_links=main_soup.find(attrs={'id': 'listing-index'})

                #kivesszük a linkeket és megvizsgáljuk
                for url in list_links.find_all('a'):
                    link=url.get('href')
                    if link not in links and "https://www.alberlet.hu/kiado_alberlet" in link:
                        links.append(link)
                        print(link)
                #print("-")
        else:
            continue
            print("Status code: SHIT!!! ", "Page:", page, "-%s seconds-" % (time.time() - start_time))
    except requests.exceptions.Timeout:
        print("TimeOutError")
        continue
        
print(len(links), "-%s seconds -"% (time.time() - start_time))

# Végig megy a kigyűjtött linkeken

In [ ]:
headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
albi_lista=[]
start_time=time.time()

for link in links:      
    
    try:
        r_sub=requests.get(link, timeout=5, headers=headers)
        if r_sub.status_code == 200:
            soup=BeautifulSoup(r_sub.content, 'html.parser')
            cnt=1
            key=[]#a táblázatból külön vesszük az oszlop nevet és a tartalmát
            val=[]          
            try:
                rows=soup.find('table', {"class", "profile-table"}).find_all("td")
                for i in range(len(rows)):
                    elem=rows[i].text.strip().replace("\n", "").replace("Árfigyelés", "").replace("Árcsökkenés", "").replace("HUF / hó", "").replace("HUF", "")
                    if cnt%2!=0:
                        key.append(elem)
                    else:
                        val.append(elem)
                    cnt+=1
                table=dict(zip(key, val))#dict-et csinálunk belőle mert a táblázat tartalma nem ugyan az minden esetben

                city=soup.find("span", {"class","profile__header-item profile__header-address"}).text.strip().split(",")[0]
                district=soup.find("span", {"class","profile__header-item profile__header-address"}).text.strip().split(",")[1]
                street=soup.find("h1", {"class","profile__header-item profile__header-address profile__header-street"}).text.strip()
                table["Város"]=city
                table["Kerület"]=district
                table["Utca"]=street
                table["Link"]=link
                
                albi_lista.append(table)
                print(len(albi_lista), "-%s seconds-" % (time.time() - start_time))

            except AttributeError:
                print("AttributeError")
                continue

        else:
            print("Hibás status code")
    except requests.exceptions.Timeout:
        print("TimeOutError")
        continue

        
print(len(albi_lista), "-%s seconds-" % (time.time() - start_time))




# DataFrame-é alakitom

In [ ]:
df=pd.DataFrame(data=albi_lista)

albi_df=df[['Épület típusa', 'Bérleti díj', 'Kaució', 
             'Rezsiköltség', 'Közös költség','Méret',
             'Szobák száma', 'Különnyíló szoba',
             'Berendezés', 'Emelet', 'Lift',
             'Kilátás', 'Minimum bérlési idő',
             'Beköltözhető', 'Város',
             'Kerület', 'Utca', 'Link' , 'Új bérleti díj']]

albi_df["Kaució"]=albi_df["Kaució"].str.strip()
albi_df["Kaució"]=albi_df["Kaució"].str.strip('\n')
albi_df["Kaució"]=albi_df["Kaució"].str.replace(" ", "")
albi_df["Kaució"]= albi_df["Kaució"].fillna(0)
albi_df["Kaució"]=albi_df["Kaució"].astype(int)

albi_df["Bérleti díj"] = albi_df["Bérleti díj"].fillna(albi_df["Új bérleti díj"])
albi_df["Bérleti díj"]=albi_df["Bérleti díj"].str.strip()
albi_df["Bérleti díj"]=albi_df["Bérleti díj"].str.strip('\n')
albi_df["Bérleti díj"]=albi_df["Bérleti díj"].str.replace(" ", "")
albi_df["Bérleti díj"]=albi_df["Bérleti díj"].astype(int)

albi_df["Rezsiköltség"]=albi_df["Rezsiköltség"].str.strip()
albi_df["Rezsiköltség"]=albi_df["Rezsiköltség"].str.strip('\n')
albi_df["Rezsiköltség"]=albi_df["Rezsiköltség"].str.replace(" ", "")
albi_df["Rezsiköltség"]= albi_df["Rezsiköltség"].fillna(0)
albi_df["Rezsiköltség"]=albi_df["Rezsiköltség"].astype(int)

albi_df["Közös költség"]=albi_df["Közös költség"].str.strip()
albi_df["Közös költség"]=albi_df["Közös költség"].str.strip('\n')
albi_df["Közös költség"]=albi_df["Közös költség"].str.replace(" ", "")
albi_df["Közös költség"]=albi_df["Közös költség"].str.replace("Nincs", "0")
albi_df["Közös költség"]=albi_df["Közös költség"].str.replace("Bérletidíjtartalmazza", "0")
albi_df["Közös költség"]= albi_df["Közös költség"].fillna(0)
albi_df["Közös költség"]=albi_df["Közös költség"].astype(int)

albi_df["Budapest_lat"]="47,4983815"
albi_df["Budapest_long"]="19.0404707"

albi_df["Total ktg"]=albi_df["Bérleti díj"]+albi_df["Rezsiköltség"]+albi_df["Közös költség"]



albi_df.sort_values(by='Total ktg', ascending=True)

# Import

In [ ]:
date=datetime.datetime.today().strftime('%Y-%m-%d')

albi_df.to_csv("albi_df"+date+".csv", encoding='utf-8')

In [ ]:
albi_df.to_excel("albi_df"+date+".xlsx")